<a href="https://colab.research.google.com/github/gergrgerg/chatbot/blob/main/Copy_of_train_textbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
#importing necessary libraries
!pip install pyMuPDF
import urllib.request
import fitz
import re
import numpy as np
import tensorflow_hub as hub

import os
from sklearn.neighbors import NearestNeighbors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 20.3 MB/s eta 0:00:00


In [23]:
# defining the required functions
def download_pdf(url, output_path):
    urllib.request.urlretrieve(url, output_path)


def preprocess(text):
    text = text.replace('\n', ' ')
    text = re.sub('\s+', ' ', text)
    return text


def pdf_to_text(path, start_page=1, end_page=None):
    doc = fitz.open(path)
    total_pages = doc.page_count

    if end_page is None:
        end_page = total_pages

    text_list = []

    for i in range(start_page-1, end_page):
        text = doc.load_page(i).get_text("text")
        text = preprocess(text)
        text_list.append(text)

    doc.close()
    return text_list


def text_to_chunks(texts, word_length=150, start_page=1):
    text_toks = [t.split(' ') for t in texts]
    page_nums = []
    chunks = []

    for idx, words in enumerate(text_toks):
        for i in range(0, len(words), word_length):
            chunk = words[i:i+word_length]
            if (i+word_length) > len(words) and (len(chunk) < word_length) and (
                len(text_toks) != (idx+1)):
                text_toks[idx+1] = chunk + text_toks[idx+1]
                continue
            chunk = ' '.join(chunk).strip()
            chunk = f'[{idx+start_page}]' + ' ' + '"' + chunk + '"'
            chunks.append(chunk)
    return chunks

In [24]:
# loading the textbook
download_pdf('https://www.gopalancolleges.com/gpuc/pdf/pu-2-science/II%20PUC%20Eng%20TB.pdf','book.pdf')
#converting into .txt format
pages=pdf_to_text('book.pdf')
#creating chunks from the text generated
chunks=text_to_chunks(pages)

In [ ]:
total_words=0
for page in pages:
  total_words +=len(page.split(' '))

print("the toalwords are ", total_words)
print("length of chunks is",len(chunks))

the toalwords are  72430
length of chunks is 483


In [25]:
#displaying random number of chunks
import random
random.seed(123)
random_idxs = random.choices(range(0, len(chunks)), k=5)

for idx in random_idxs:
  print(chunks[idx])
  print()

[127] "2016                                                                                                         2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16"

[127] "2016                                                                                                         2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16"

[127] "2016                                                                                                         2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16  2015-16"

[127] "2016                 

In [26]:
#defining the class for creation of vector space
class SemanticSearch:

    def __init__(self):
        self.use = hub.load('https://tfhub.dev/google/universal-sentence-encoder/4')
        self.fitted = False


    def fit(self, data, batch=1000, n_neighbors=5):
        self.data = data
        self.embeddings = self.get_text_embedding(data, batch=batch)
        n_neighbors = min(n_neighbors, len(self.embeddings))
        self.nn = NearestNeighbors(n_neighbors=n_neighbors)
        self.nn.fit(self.embeddings)
        self.fitted = True


    def __call__(self, text, return_data=True):
        inp_emb = self.use([text])
        neighbors = self.nn.kneighbors(inp_emb, return_distance=False)[0]

        if return_data:
            return [self.data[i] for i in neighbors]
        else:
            return neighbors


    def get_text_embedding(self, texts, batch=1000):
        embeddings = []
        for i in range(0, len(texts), batch):
            text_batch = texts[i:(i+batch)]
            emb_batch = self.use(text_batch)
            embeddings.append(emb_batch)
        embeddings = np.vstack(embeddings)
        return embeddings

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
recommender = SemanticSearch()
recommender.fit(chunks)

In [30]:
recommender('what is ')


NameError: name 'recommender' is not defined